# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
slice_index = int(num_test/2)
test_data, test_labels = newsgroups_test.data[slice_index:], newsgroups_test.target[slice_index:]
dev_data, dev_labels = newsgroups_test.data[:slice_index], newsgroups_test.target[:slice_index]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print ('training label shape:', train_labels.shape)
print ('test label shape:', test_labels.shape)
print ('dev label shape:', dev_labels.shape)
print ('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [3]:
def P1(num_examples=5):
### STUDENT START ###
    
    for index in range(num_examples):
        print("\nTraining Label: ", train_labels[index] , '(',newsgroups_train.target_names[train_labels[index]],')')
        print("\nTraining Text:\n\n", train_data[index])
        

### STUDENT END ###
P1(5)


Training Label:  1 ( comp.graphics )

Training Text:

 Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Training Label:  3 ( talk.religion.misc )

Training Text:

 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(type(X))
print(X.shape)
print(X)
print(X.toarray())  
print(X.toarray().shape)  



['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
<class 'scipy.sparse.csr.csr_matrix'>
(4, 9)
  (0, 1)	1
  (0, 2)	1
  (0, 6)	1
  (0, 3)	1
  (0, 8)	1
  (1, 5)	1
  (1, 1)	2
  (1, 6)	1
  (1, 3)	1
  (1, 8)	1
  (2, 4)	1
  (2, 7)	1
  (2, 0)	1
  (2, 6)	1
  (2, 3)	1
  (2, 8)	1
  (3, 1)	1
  (3, 2)	1
  (3, 6)	1
  (3, 3)	1
  (3, 8)	1
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
(4, 9)


In [5]:

vectorizer = CountVectorizer()
vector_train_data = vectorizer.fit_transform(train_data)



## a
print('Size of the vocabulary: ', vector_train_data.shape) 

nnz = vector_train_data.nnz           # Number of Non Zero entries for features
noe = vector_train_data.shape[0]      # Number of Examples
nof = vector_train_data.shape[1]      # Number of Features

print('Average number of non-zero features per example: {0:.2f}. '.format(nnz/noe))
print('Fraction of the entries in the matrix are non-zero: {0:.5f}.'.format(nnz / (noe * nof)))

## b
print('0th and last feature strings: ', vectorizer.get_feature_names()[0],'&', vectorizer.get_feature_names()[-1])

## c

vocabulury = ["atheism", "graphics", "space", "religion"]
vector_train_data = vectorizer.fit_transform(vocabulury)
print('Size of the vocabulary: ', vector_train_data.shape) 

nnz = vector_train_data.nnz           # Number of Non Zero entries for features
noe = vector_train_data.shape[0]      # Number of Examples
nof = vector_train_data.shape[1]      # Number of Features

print(vectorizer.get_feature_names())
print(vector_train_data.toarray())

print('Average number of non-zero features per example: {0:.2f}. '.format(nnz/noe))
print('Fraction of the entries in the matrix are non-zero: {0:.5f}.'.format(nnz / (noe * nof)))

## d


vectorizer = CountVectorizer(analyzer = 'char',ngram_range=(1,3))
vector_train_data = vectorizer.fit_transform(train_data)

print('Size of the vocabulary: ', vector_train_data.shape) 

nnz = vector_train_data.nnz           # Number of Non Zero entries for features
noe = vector_train_data.shape[0]      # Number of Examples
nof = vector_train_data.shape[1]      # Number of Features

print('Average number of non-zero features per example: {0:.2f}. '.format(nnz/noe))
print('Fraction of the entries in the matrix are non-zero: {0:.5f}.'.format(nnz / (noe * nof)))

## e


vectorizer = CountVectorizer(min_df=10)
vector_train_data = vectorizer.fit_transform(train_data)

print('Size of the vocabulary: ', vector_train_data.shape) 

nnz = vector_train_data.nnz           # Number of Non Zero entries for features
noe = vector_train_data.shape[0]      # Number of Examples
nof = vector_train_data.shape[1]      # Number of Features

print('Average number of non-zero features per example: {0:.2f}. '.format(nnz/noe))
print('Fraction of the entries in the matrix are non-zero: {0:.5f}.'.format(nnz / (noe * nof)))


## f

vectorizer = CountVectorizer()
vector_train_data = vectorizer.fit_transform(train_data)
vector_dev_data = vectorizer.fit_transform(dev_data)

print('Size of the vocabulary: ', vector_train_data.shape) 

nof_train = vector_train_data.shape[1]      # Number of Features
nof_dev = vector_dev_data.shape[1]      # Number of Features


print(nof_train,nof_dev)

Size of the vocabulary:  (2034, 26879)
Average number of non-zero features per example: 96.71. 
Fraction of the entries in the matrix are non-zero: 0.00360.
0th and last feature strings:  00 & zyxel
Size of the vocabulary:  (4, 4)
['atheism', 'graphics', 'religion', 'space']
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]
Average number of non-zero features per example: 1.00. 
Fraction of the entries in the matrix are non-zero: 0.25000.
Size of the vocabulary:  (2034, 35557)
Average number of non-zero features per example: 717.30. 
Fraction of the entries in the matrix are non-zero: 0.02017.
Size of the vocabulary:  (2034, 3064)
Average number of non-zero features per example: 72.68. 
Fraction of the entries in the matrix are non-zero: 0.02372.
Size of the vocabulary:  (2034, 26879)
26879 16246


In [6]:
vectorizer.get_feature_names()[-10000]

'fight'

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [7]:
#def P2():
### STUDENT START ###


### STUDENT END ###
#P2()

(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [8]:
def P3():
    
### STUDENT START ###
    # Vectorize the inputs
    vectorizer = CountVectorizer()
    vectorizer.fit(train_data)
    vector_train_data = vectorizer.transform(train_data)
    vector_dev_data = vectorizer.transform(dev_data)
    
    #-------------------------------------------------------------------------
    # k nearest neighbors classifier
    #-------------------------------------------------------------------------
    k_range = range(1,vector_dev_data.shape[0]+1)
    n_neighbors = {'n_neighbors': np.array(k_range)}

    #Search for best value of k
    fit_model = KNeighborsClassifier()
    k_grid = GridSearchCV(fit_model, n_neighbors)
    k_grid.fit(vector_train_data, train_labels)
    best_k = k_grid.best_params_['n_neighbors']

    #Fit the model with the best k and find F1 score
    fit_model = KNeighborsClassifier(n_neighbors=best_k)
    fit_model.fit(vector_train_data, train_labels)
    predict = fit_model.predict(vector_dev_data)

    #Find the F1 Score for the best k
    f1_score = metrics.f1_score(y_true = dev_labels, y_pred = predict, average="macro")
    print('f1 score for the best k = {0} is {1}'.format(best_k,f1_score))

    #-------------------------------------------------------------------------
    # Multinomial classifier
    #-------------------------------------------------------------------------

    alphas = {'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}

    #Search for best value of alpha
    fit_model = MultinomialNB()
    mn_grid = GridSearchCV(fit_model, alphas)
    mn_grid.fit(vector_train_data, train_labels)
    best_alpha = mn_grid.best_params_['alpha']


    #Fit the model with the best alpha and find F1 score
    fit_model = MultinomialNB(alpha = best_alpha)
    fit_model.fit(vector_train_data, train_labels)
    predict = fit_model.predict(vector_dev_data)

    #Find the F1 Score for the best alpha
    f1_score = metrics.f1_score(y_true = dev_labels, y_pred = predict, average="macro")
    print('f1 score for the best alpha = {0} is {1}'.format(best_alpha,f1_score))

  
    
    #-------------------------------------------------------------------------
    # Logistics Regressor with penalty 12
    #-------------------------------------------------------------------------
    Cs = [0.0001, 0.001, 0.01, 0.1, 1, 2, 5, 10]
    c_list = {'C': Cs}

    #Search for best value of C
    fit_model = LogisticRegression(penalty = 'l2')
    mn_grid = GridSearchCV(fit_model, c_list)
    mn_grid.fit(vector_train_data, train_labels)
    best_c = mn_grid.best_params_['C']


    #Fit the model with the best C and find F1 score
    fit_model = LogisticRegression(penalty = 'l2', C = best_c)
    fit_model.fit(vector_train_data, train_labels)
    predict = fit_model.predict(vector_dev_data)

    #Find the F1 Score for the best C
    f1_score = metrics.f1_score(y_true = dev_labels, y_pred = predict, average="macro")
    print('f1 score for the best C = {0} is {1}'.format(best_c,f1_score))
    
    #Sum of squared  weight for each C
    print('\n')
    for c in Cs:
        fit_model = LogisticRegression(penalty = 'l2', C = c)
        fit_model.fit(vector_train_data, train_labels)
        predict = fit_model.predict(vector_dev_data)
        print('Sum of Squared Weight : {0} for C value:  {1}'.format(np.sum(fit_model.coef_**2, axis=1), c))
    
### STUDENT END ###
P3()

f1 score for the best k = 151 is 0.4028149344405508
f1 score for the best alpha = 0.01 is 0.7552894198013128
f1 score for the best C = 0.1 is 0.6794337099161509


Sum of Squared Weight : [0.00770175 0.0119412  0.00943508 0.00910284] for C value:  0.0001
Sum of Squared Weight : [0.16509345 0.20095275 0.18067094 0.18724278] for C value:  0.001
Sum of Squared Weight : [2.54139589 2.93970937 2.86246884 2.25002868] for C value:  0.01
Sum of Squared Weight : [27.13465361 24.65876281 27.45839548 23.0209231 ] for C value:  0.1
Sum of Squared Weight : [166.9976657  130.9242447  158.02131132 145.71535688] for C value:  1
Sum of Squared Weight : [257.67095549 197.97941925 239.93584167 226.66760255] for C value:  2
Sum of Squared Weight : [423.25533245 322.34190104 389.76882891 378.22011225] for C value:  5
Sum of Squared Weight : [586.60932474 449.24063855 540.63301964 530.88094065] for C value:  10


ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [9]:
def P4(ngram=1):
### STUDENT START ###
    best_c = 0.1
    if ngram==1:
        vectorizer = CountVectorizer()
    else:
        vectorizer = CountVectorizer(ngram_range=(ngram, ngram))
        
    vectorizer.fit(train_data)
    vector_train_data = vectorizer.transform(train_data)
    vector_dev_data = vectorizer.transform(dev_data)

    fit_model = LogisticRegression(penalty = 'l2', C = best_c)
    fit_model.fit(vector_train_data, train_labels)

    sorted_index = np.argsort(np.absolute(fit_model.coef_), axis = 1)
    large_wt_features = 5
    feature_names = np.array(vectorizer.get_feature_names())


    #Find the features with the top weight for each class.
    print( "Top 5 weighted features per class:")
    for cls in range(fit_model.coef_.shape[0]):
        list_target_name = []
        print('\n',newsgroups_train.target_names[cls], ':')
        for feature in sorted_index[cls, -large_wt_features:fit_model.coef_.shape[1]]:
            list_target_name.append(feature_names[feature])
        print(list_target_name)


    # Display the corresponding weights for each feature with top weights in each class
    grid = newsgroups_train.target_names
    grid = np.hstack(('Features/Class', grid))
    print('\n Weights of the each feature for each class for top 5 Weights:\n')
    for cls in range(fit_model.coef_.shape[0]):
        for feature in sorted_index[cls, -large_wt_features:fit_model.coef_.shape[1]]:
            rounded_weights = np.round(fit_model.coef_[:, feature], 5)
            row = np.hstack((vectorizer.get_feature_names()[feature], rounded_weights))
            grid = np.vstack((grid, row))
    print (grid)


### STUDENT END ###

P4()
print("For Bigram Features: ")
P4(ngram=2)

Top 5 weighted features per class:

 alt.atheism :
['atheists', 'bobby', 'religion', 'atheism', 'space']

 comp.graphics :
['computer', 'file', 'image', 'space', 'graphics']

 sci.space :
['god', 'nasa', 'orbit', 'graphics', 'space']

 talk.religion.misc :
['order', 'blood', 'christians', 'christian', 'space']

 Weights of the each feature for each class for top 5 Weights:

[['Features/Class' 'alt.atheism' 'comp.graphics' 'sci.space'
  'talk.religion.misc']
 ['atheists' '0.46116' '-0.07943' '-0.15839' '-0.29529']
 ['bobby' '0.47826' '-0.12039' '-0.16785' '-0.2278']
 ['religion' '0.494' '-0.29879' '-0.39323' '0.00377']
 ['atheism' '0.49543' '-0.20727' '-0.19991' '-0.26775']
 ['space' '-0.65524' '-0.71405' '1.25881' '-0.59016']
 ['computer' '-0.0398' '0.55901' '-0.32905' '-0.22873']
 ['file' '-0.17738' '0.64122' '-0.42158' '-0.28826']
 ['image' '-0.26355' '0.6421' '-0.36763' '-0.21602']
 ['space' '-0.65524' '-0.71405' '1.25881' '-0.59016']
 ['graphics' '-0.41119' '1.0075' '-0.65112' '-0.

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [50]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
    
### STUDENT START ###
    s = s.lower()                          # Lower case
    s = re.sub(r'\b\d+\b', " numeric ", s) # Replace with single token
    s = re.sub(r'\W+', " ", s)             # Remove Special characters  
    s = re.sub(r'_', " ", s)               # Remove underscore _
    
    # Shorten the long words to 5 letter word
    words = s.split(" ")
    s=""
    for word in words: s= s + " " + word[:5]
    return s
### STUDENT START ###



def P5(preprocessor=None, stop_words=None):
    
### STUDENT START ###
    
    vectorizer = CountVectorizer(preprocessor=preprocessor, stop_words=stop_words)
    vectorizer.fit(train_data)
    vector_train_data = vectorizer.transform(train_data)
    vector_dev_data = vectorizer.transform(dev_data)
    dict_size = len(vectorizer.get_feature_names())
    
   
    fit_model = LogisticRegression()
    fit_model.fit(vector_train_data, train_labels)
    predict = fit_model.predict(vector_dev_data)
 
    #Find the F1 Score for the best k
    f1_score = metrics.f1_score(y_true = dev_labels, y_pred = predict, average="macro")
    print('\nDictionary size with {0} is {1}'.format(preprocessor.__name__,dict_size))
    print('f1 score for the {0} is {1}'.format(preprocessor.__name__,f1_score))
    return f1_score , dict_size
    
### STUDENT END ###
f1_score_empty , dict_size_empty = P5(preprocessor = empty_preprocessor)
f1_score_better , dict_size_better = P5(preprocessor = better_preprocessor, stop_words = 'english')
print('\nF1 score improved by: {0}'.format(f1_score_better-f1_score_empty))
print('Dictionary Size reduced by: {0}'.format(dict_size_empty-dict_size_better))




Dictionary size with empty_preprocessor is 33291
f1 score for the empty_preprocessor is 0.6870608946268841

Dictionary size with better_preprocessor is 14842
f1 score for the better_preprocessor is 0.7172353984875182

F1 score improved by: 0.030174503860634028
Dictionary Size reduced by: 18449


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance